# <span style='color:#ff5f27'> 👨🏻‍🏫 BigQuery Online External Feature Group Creation</span>

Follow this [guide](https://docs.hopsworks.ai/3.0/user_guides/fs/storage_connector/creation/bigquery/) to set up a connection to BigQuery.

In addition, you can read about [External Feature Groups](https://docs.hopsworks.ai/3.0/user_guides/fs/feature_group/create_external/).

## <span style='color:#ff5f27'> 🔮 Retrieve a Connector

Firstly, connect to feature store and then retrieve your **BigQuery storage connector**.

Make sure that your storage connector has the **BigQuery Query** read options.

![big_query](images/big_query.png)

Replace `my_storage_connector_name` with your BigQuery storage connector name.

In [ ]:
import hopsworks

# Connect to the Hopsworks feature store
project = hopsworks.login()

# Retrieve the feature store
feature_store = project.get_feature_store()

# Retrieve storage connector
connector = feature_store.get_storage_connector('my_storage_connector_name')

## <span style='color:#ff5f27'> 📝 Read Data </span>

You can retrieve your data by passing a SQL query as a string into **query** parameter of the `.read()` method.

To indicate what data you need, follow the next table format:

`{dataset_name}.{table_name}`

In [ ]:
query_str = "SELECT * FROM {dataset_name}.{table_name}"

# read results from a SQL 
df = connector.read(query=query_str)    

> **.read()** method returns a PySpark DataFrame.

In [ ]:
type(df)

In [ ]:
df.show(5)

## <span style='color:#ff5f27'> ⚙️ Online Feature Group Creation</span>

To create an **Online External Feature Group**, use `feature_store.create_external_feature_group()` method.

Provide the parameters for your query and the BQ storage connector:
- `query` - your SQL query string. 
- `storage_connector` - your **BigQuery Storage Connector**.
- `online_enabled` - set to True.

In [ ]:
my_external_fg = feature_store.create_external_feature_group(
    name="name_external_fg",
    version=1,
    description="Description of the Online External Big Query Table",
    primary_key=['pk1_name','pk2_name'],
    event_time='event_time_col_name',
    query=query_str,
    storage_connector=connector,
    online_enabled=True
)

Don't forget to register a feature group metadata object in Hopsworks!

In [ ]:
my_external_fg.save()

Now, you should be able to see your Online External Feature Group for the BQ table in the Hopsworks UI.

![BQ_table](images/big_query_table.png)

---

## <span style='color:#ff5f27'> ⬇️ Insert to online storage</span>

You can enable online storage for external feature groups, however, the sync from the external storage to Hopsworks online storage needs to be done manually, so you need to acomplish the next steps:
- Filter data to sync to online.
- Read from external storage.
- Insert to online storage.

Replace `column_name` and `filter_value` with required **column name** and a **value** by which you will filter.

In [ ]:
column_name = 'column_name'
filter_value = 'filter_value'

In [ ]:
df = my_external_fg.filter(my_external_fg[column_name] == filter_value).read()

df.show(1)

Insert filtered data to online storage in order to be able to call `.get_feature_vector()` method.

In [ ]:
my_external_fg.insert(df, wait=True)

## <span style='color:#ff5f27'> ⚙️ Feature View Creation</span>

In [ ]:
query = my_external_fg.select_all()

In [ ]:
feature_view = feature_store.get_or_create_feature_view(
    name='my_feature_view_name',
    version=1,
    query=query
)

In [ ]:
vector = feature_view.get_feature_vector(
    entry = {"pk1_name": 1, "pk2_name": 2}
)
vector

---